In [1]:
import pandas as pd
import os

In [2]:
num_files = len(os.listdir('spread_probability'))

In [3]:
num_files

217

In [4]:
df = pd.read_csv('spread_probability/1.csv')
df

,from,to,prob
0,10010201001,10010201001,1.032443e-02
1,10010201001,10010201002,1.664969e-02
2,10010201001,10010202001,9.542054e-03
3,10010201001,10010202002,1.214415e-02
4,10010201001,10010203001,3.202022e-02
...,...,...,...
4977097,10599734002,390250404011,1.451897e-06
4977098,10599734002,471870505022,7.259484e-07
4977099,10599734002,471870505023,1.451897e-06
4977100,10599734002,471870505042,2.177845e-06


In [5]:
src_cbgs = df['from'].unique().tolist()
grouped_src = df.groupby('from')
src_cbg_df = grouped_src.get_group(src_cbgs[0]).sort_values(by='prob', ascending=False).reset_index(drop=True)
src_cbg_df = src_cbg_df.iloc[:30]
sum_prob = src_cbg_df['prob'].sum()
src_cbg_df['prob'] = src_cbg_df['prob']/sum_prob
src_cbg_df

,from,to,prob
0,10010201001,10010205002,0.116397
1,10010201001,10010208022,0.067747
2,10010201001,10010208012,0.061990
3,10010201001,10010203001,0.052379
4,10010201001,10010208021,0.049683
5,10010201001,10010206001,0.046656
6,10010201001,10010205003,0.046233
7,10010201001,10510313001,0.040068
8,10010201001,10010205001,0.039182
9,10010201001,10010204002,0.032063


In [6]:
def cut_each_src(src_cbg_df):
    src_cbg_df = src_cbg_df.sort_values(by='prob', ascending=False).reset_index(drop=True)
    src_cbg_df = src_cbg_df.iloc[:30]
    sum_prob = src_cbg_df['prob'].sum()
    src_cbg_df['prob'] = src_cbg_df['prob']/sum_prob
    return src_cbg_df

In [7]:
os.mkdir('spread_probability_top30')

In [8]:
res = []

for i in range(num_files):
    df = pd.read_csv('spread_probability/%d.csv'%(i+1))
    src_cbgs = df['from'].unique().tolist()
    grouped_src = df.groupby('from')

    for src_cbg in src_cbgs:
        res.append(cut_each_src(grouped_src.get_group(src_cbg)))

    print(i, 'done')

    if (i+1) %50 == 0:
        pd.concat(res).to_csv('spread_probability_top30/%d.csv'%((i+1)//50), index=False)
        res = []
        
pd.concat(res).to_csv('spread_probability_top30/{}.csv'.format(num_files// 50+1), index=False)

0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done
116 done
117 done
118 done
119 done
120 done
121 done
122 done
123

In [9]:
df = pd.read_csv('spread_probability_top30/1.csv')
df

,from,to,prob
0,10010201001,10010205002,0.116477
1,10010201001,10010208022,0.067741
2,10010201001,10010208012,0.061967
3,10010201001,10010203001,0.052342
4,10010201001,10010208021,0.049671
...,...,...,...
1498267,121030267012,121030245122,0.023506
1498268,121030267012,121030269084,0.023476
1498269,121030267012,121030263002,0.023441
1498270,121030267012,120950171031,0.023414


In [10]:
df['from'] = df['from'].apply(lambda x: int(('%012d'%x)[:5]))
df['to'] = df['to'].apply(lambda x: int(('%012d'%x)[:5]))
df

,from,to,prob
0,1001,1001,0.116477
1,1001,1001,0.067741
2,1001,1001,0.061967
3,1001,1001,0.052342
4,1001,1001,0.049671
...,...,...,...
1498267,12103,12103,0.023506
1498268,12103,12103,0.023476
1498269,12103,12103,0.023441
1498270,12103,12095,0.023414


In [11]:
from_county = df['from'].unique().tolist()
grouped_from_county = df.groupby('from')
from_df = grouped_from_county.get_group(from_county[0])
from_df

,from,to,prob
0,1001,1001,0.116477
1,1001,1001,0.067741
2,1001,1001,0.061967
3,1001,1001,0.052342
4,1001,1001,0.049671
...,...,...,...
955,1001,1101,0.019801
956,1001,1001,0.018793
957,1001,1001,0.016030
958,1001,1001,0.015814


In [12]:
from_df['prob'].sum()

31.999999999999957

In [13]:
from_c = from_df['from'].tolist()[0]
from_df = from_df.groupby('to')['prob'].sum().reset_index()
from_df['from'] = len(from_df) * [from_c]
from_df['prob'] = from_df['prob']/from_df['prob'].sum()
from_df = from_df[['from','to','prob']]

In [14]:
def cal_county(from_df, county):
    from_df = from_df.groupby('to')['prob'].sum().reset_index()
    from_df['from'] = len(from_df) * [county]
    from_df['prob'] = from_df['prob']/from_df['prob'].sum()
    from_df = from_df[['from','to','prob']]
    return from_df

In [15]:
res = []
for i in range(1,5):
    df = pd.read_csv('spread_probability_top30/%d.csv'%i)
    df['from'] = df['from'].apply(lambda x: int(('%012d'%x)[:5]))
    df['to'] = df['to'].apply(lambda x: int(('%012d'%x)[:5]))
    from_county = df['from'].unique().tolist()
    grouped_from_county = df.groupby('from')

    for county in from_county:
        res.append(cal_county(grouped_from_county.get_group(county),county))
res = pd.concat(res)
res

,from,to,prob
0,1001,1001,0.691617
1,1001,1021,0.027110
2,1001,1047,0.041190
3,1001,1051,0.222386
4,1001,1073,0.003892
...,...,...,...
15,50021,48453,0.000513
16,50021,50001,0.111022
17,50021,50007,0.001666
18,50021,50021,0.827989


In [16]:
res.to_csv('spread_probability_top30/county_level.csv', index=False)